In [303]:
from enum import Enum, auto
from pydantic import BaseModel
from typing import Dict, Any, Optional
from typing_extensions import Protocol, runtime_checkable
from result import Result, Ok, Err
from app.variable.expr import LazyExpr
import ast


In [304]:
# imp = ["from datetime import datetime", "from typing import List"]
imp = []

# s = "lambda x: all(isinstance(i, datetime) for i in x) if isinstance(x, list) else False"
s = "(aaaaa := 1)"
# s = """
# 1+1
# """
f = LazyExpr(s, imp)

# print(ast.dump(f._ast, indent=2))
print(f.unbound)
print(f.target)

f.eval(env={"a": 1024})

set()
aaaaa


1